In [22]:
import mysql.connector
import tabulate

In [23]:
def connect():
    #mysql -h 34.45.141.24 --port 12373 -u jonasxie-ro -p2963274fe6f86749b5ddd9739028aaec

    conn = mysql.connector.connect(
        host = "34.45.141.24",
        port = 12373,
        user = "jonasxie-ro",
        password = "2963274fe6f86749b5ddd9739028aaec"
    )

    cursor = conn.cursor()
    return conn, cursor

def conn_close(conn, cursor):
    cursor.close()
    conn.close()

In [24]:
def show_sql(cursor):
    rows = cursor.fetchall()
    try:
        columns = [i[0] for i in cursor.description]
    except:
        return None
    print(tabulate.tabulate(rows, headers=columns, tablefmt='psql')) 


def sql_exc(query:str, cursor):
    try:
        cursor.execute(query)
    except Exception as e:
        print(e)
        if e.errno == 1065:
            print("No query")
        if e.errno == 2013:
            print("Reconnect")
            return
        elif e.errno == 2014:
            print("Unread results")
            show_sql(cursor)
        return
    try:
        show_sql(cursor)
    except Exception as e:
        return
    

In [25]:
conn, cursor = connect()

In [26]:
# cursor.execute("SHOW DATABASES")
sql_exc("USE ro_recipes", cursor)
sql_exc("SHOW TABLES", cursor)


+------------------------+
| Tables_in_ro_recipes   |
|------------------------|
| Ingredient_Classes     |
| Ingredients            |
| Measurements           |
| Recipe_Classes         |
| Recipe_Ingredients     |
| Recipes                |
+------------------------+


In [27]:
sql_exc("SHOW CREATE TABLE Ingredients", cursor)

+-------------+------------------------------------------------------------------------------------------------+
| Table       | Create Table                                                                                   |
|-------------+------------------------------------------------------------------------------------------------|
| Ingredients | CREATE TABLE `Ingredients` (                                                                   |
|             |   `IngredientID` int NOT NULL DEFAULT '0',                                                     |
|             |   `IngredientName` varchar(255) CHARACTER SET utf8mb3 COLLATE utf8mb3_general_ci DEFAULT NULL, |
|             |   `IngredientClassID` smallint DEFAULT '0',                                                    |
|             |   `MeasureAmountID` smallint DEFAULT '0',                                                      |
|             |   PRIMARY KEY (`IngredientID`)                                                  

In [28]:
sql_exc("""
        SELECT r.RecipeTitle, r.RecipeClassID 
        FROM Recipes r
        JOIN Recipe_Classes rc ON r.RecipeClassID = rc.RecipeClassID
        WHERE rc.RecipeClassDescription IN ('Vegetable', 'Salad')
        """, cursor)

+---------------------+-----------------+
| RecipeTitle         |   RecipeClassID |
|---------------------+-----------------|
| Garlic Green Beans  |               2 |
| Asparagus           |               2 |
| Mike's Summer Salad |               4 |
+---------------------+-----------------+


In [29]:
sql_exc("""
        SELECT r.RecipeID, r.RecipeTitle 
        FROM Recipes r
        JOIN Recipe_Ingredients ri ON r.RecipeID = ri.RecipeID
        JOIN Ingredients i ON ri.IngredientID = i.IngredientID
        JOIN Ingredient_Classes ic ON i.IngredientClassID = ic.IngredientClassID
        WHERE ic.IngredientClassDescription = 'Seafood'
        LIMIT 50
        """, cursor)

+------------+-------------------------------------------------------+
|   RecipeID | RecipeTitle                                           |
|------------+-------------------------------------------------------|
|         14 | Salmon Filets in Parchment Paper                      |
|         11 | Huachinango Veracruzana (Red Snapper, Veracruz style) |
+------------+-------------------------------------------------------+


In [30]:
sql_exc("""
        SELECT * FROM Recipes WHERE RecipeID IN (
        SELECT RecipeID 
        FROM Recipe_Ingredients 
        WHERE IngredientID IN (
                SELECT IngredientID 
                FROM Ingredients 
                WHERE IngredientName = 'Beef')) 
        AND RecipeID IN (
        SELECT RecipeID 
        FROM Recipe_Ingredients 
        WHERE IngredientID IN (
                SELECT IngredientID 
                FROM Ingredients 
                WHERE IngredientName = 'Garlic')) 
        AND RecipeID NOT IN (
        SELECT RecipeID 
        FROM Recipe_Ingredients 
        WHERE IngredientID IN (
                SELECT IngredientClassID 
                FROM Ingredient_Classes 
                WHERE IngredientClassDescription = 'Seafood'))
        """, cursor)

+------------+---------------+-----------------+---------------+---------+
| RecipeID   | RecipeTitle   | RecipeClassID   | Preparation   | Notes   |
|------------+---------------+-----------------+---------------+---------|
+------------+---------------+-----------------+---------------+---------+


In [31]:
conn, cursor = connect()

In [32]:
sql_exc("USE ro_company1", cursor)
sql_exc("SHOW TABLES", cursor)

+-------------------------+
| Tables_in_ro_company1   |
|-------------------------|
| account                 |
| branch                  |
| business                |
| customer                |
| department              |
| employee                |
| individual              |
| officer                 |
| product                 |
| product_type            |
| transaction             |
+-------------------------+


In [33]:
sql_exc("""
        SELECT e.emp_id, e.fname, e.lname, e.dept_id 
        FROM employee e
        JOIN department d ON e.dept_id = d.dept_id
        WHERE d.name = 'Operations'
        """, cursor)

+----------+----------+-----------+-----------+
|   emp_id | fname    | lname     |   dept_id |
|----------+----------+-----------+-----------|
|        4 | Susan    | Hawthorne |         1 |
|        6 | Helen    | Fleming   |         1 |
|        7 | Chris    | Tucker    |         1 |
|        8 | Sarah    | Parker    |         1 |
|        9 | Jane     | Grossman  |         1 |
|       10 | Paula    | Roberts   |         1 |
|       11 | Thomas   | Ziegler   |         1 |
|       12 | Samantha | Jameson   |         1 |
|       13 | John     | Blake     |         1 |
|       14 | Cindy    | Mason     |         1 |
|       15 | Frank    | Portman   |         1 |
|       16 | Theresa  | Markham   |         1 |
|       17 | Beth     | Fowler    |         1 |
|       18 | Rick     | Tulman    |         1 |
+----------+----------+-----------+-----------+


In [34]:
sql_exc("SELECT * FROM business LIMIT 20", cursor)

+-----------+------------------------+------------+---------------+
|   cust_id | name                   | state_id   | incorp_date   |
|-----------+------------------------+------------+---------------|
|        10 | Chilton Engineering    | 12-345-678 | 1995-05-01    |
|        11 | Northeast Cooling Inc. | 23-456-789 | 2001-01-01    |
|        12 | Superior Auto Body     | 34-567-890 | 2002-06-30    |
|        13 | AAA Insurance Inc.     | 45-678-901 | 1999-05-01    |
+-----------+------------------------+------------+---------------+


In [35]:
sql_exc("""SELECT 
            a.account_id, 
            a.open_date, e.fname AS emp_fname,
            e.lname AS emp_lname, 
            c.address,
            c.city,
            c.state
        FROM account a 
        JOIN customer c ON a.cust_id = c.cust_id
        JOIN employee e ON a.open_emp_id = e.emp_id
        WHERE e.fname = 'Paula' AND e.lname = 'Roberts'
        """, cursor)

+--------------+-------------+-------------+-------------+---------------------+------------+---------+
|   account_id | open_date   | emp_fname   | emp_lname   | address             | city       | state   |
|--------------+-------------+-------------+-------------+---------------------+------------+---------|
|            1 | 2000-01-15  | Paula       | Roberts     | 47 Mockingbird Ln   | Lynnfield  | MA      |
|            2 | 2000-01-15  | Paula       | Roberts     | 47 Mockingbird Ln   | Lynnfield  | MA      |
|            3 | 2004-06-30  | Paula       | Roberts     | 47 Mockingbird Ln   | Lynnfield  | MA      |
|            4 | 2001-03-12  | Paula       | Roberts     | 372 Clearwater Blvd | Woburn     | MA      |
|            5 | 2001-03-12  | Paula       | Roberts     | 372 Clearwater Blvd | Woburn     | MA      |
|           17 | 2004-01-12  | Paula       | Roberts     | 29 Admiral Ln       | Wilmington | MA      |
|           27 | 2004-03-22  | Paula       | Roberts     | 287A 

In [36]:
sql_exc("""
        SELECT emp_id, fname, lname, superior_emp_id 
        FROM employee WHERE superior_emp_id IS NOT NULL
        """, cursor)    

+----------+----------+-----------+-------------------+
|   emp_id | fname    | lname     |   superior_emp_id |
|----------+----------+-----------+-------------------|
|        2 | Susan    | Barker    |                 1 |
|        3 | Robert   | Tyler     |                 1 |
|        4 | Susan    | Hawthorne |                 3 |
|        5 | John     | Gooding   |                 4 |
|        6 | Helen    | Fleming   |                 4 |
|        7 | Chris    | Tucker    |                 6 |
|        8 | Sarah    | Parker    |                 6 |
|        9 | Jane     | Grossman  |                 6 |
|       10 | Paula    | Roberts   |                 4 |
|       11 | Thomas   | Ziegler   |                10 |
|       12 | Samantha | Jameson   |                10 |
|       13 | John     | Blake     |                 4 |
|       14 | Cindy    | Mason     |                13 |
|       15 | Frank    | Portman   |                13 |
|       16 | Theresa  | Markham   |             

In [37]:
sql_exc("""
        SELECT a.account_id, a.avail_balance, c.state, b.name AS business_name
        FROM account a
        JOIN customer c ON a.cust_id = c.cust_id
        LEFT JOIN business b on a.cust_id = b.cust_id
        WHERE a.close_date IS NULL
        """, cursor)

+--------------+-----------------+---------+------------------------+
|   account_id |   avail_balance | state   | business_name          |
|--------------+-----------------+---------+------------------------|
|            1 |         1057.75 | MA      |                        |
|            2 |          500    | MA      |                        |
|            3 |         3000    | MA      |                        |
|            4 |         2258.02 | MA      |                        |
|            5 |          200    | MA      |                        |
|            7 |         1057.75 | MA      |                        |
|            8 |         2212.5  | MA      |                        |
|           10 |          534.12 | MA      |                        |
|           11 |          767.77 | MA      |                        |
|           12 |         5487.09 | MA      |                        |
|           13 |         2237.97 | NH      |                        |
|           14 |    

In [38]:
sql_exc("""
        SELECT e.emp_id, e.fname, e.lname, e.assigned_branch_id 
        FROM employee e
        JOIN branch b ON e.assigned_branch_id = b.branch_id
        WHERE b.address = '422 Maple St.'
        """, cursor)

+----------+----------+---------+----------------------+
|   emp_id | fname    | lname   |   assigned_branch_id |
|----------+----------+---------+----------------------|
|       10 | Paula    | Roberts |                    2 |
|       11 | Thomas   | Ziegler |                    2 |
|       12 | Samantha | Jameson |                    2 |
+----------+----------+---------+----------------------+


In [39]:
sql_exc("USE world", cursor)
sql_exc("SHOW TABLES", cursor)

+-------------------+
| Tables_in_world   |
|-------------------|
| city              |
| country           |
| countrylanguage   |
+-------------------+


In [40]:
sql_exc("""
        SELECT c.Name, c.CountryCode, c.Population, co.Continent
        FROM city c
        JOIN country co ON c.CountryCode = co.Code
        WHERE (c.Population > 13000 AND c.Population < 500000)
        AND co.Continent != 'North America'
        ORDER BY RAND()
        LIMIT 5
        """, cursor)

+------------+---------------+--------------+---------------+
| Name       | CountryCode   |   Population | Continent     |
|------------+---------------+--------------+---------------|
| Kirovograd | UKR           |       265000 | Europe        |
| Huai´an    | CHN           |       131149 | Asia          |
| Catanzaro  | ITA           |        96700 | Europe        |
| Tacna      | PER           |       215683 | South America |
| Qaramay    | CHN           |       197602 | Asia          |
+------------+---------------+--------------+---------------+


In [41]:
sql_exc("""
        SELECT count(*) AS count_of_cities FROM city c
        JOIN country co ON c.CountryCode = co.Code
        WHERE co.GovernmentForm = 'Constitutional Monarchy'
        """,cursor)

+-------------------+
|   count_of_cities |
|-------------------|
|               518 |
+-------------------+


In [42]:
sql_exc("""
        SELECT c.Name, c.CountryCode, c.Population, co.GovernmentForm, cl.Language As OfficialLanguage
        FROM city c
        JOIN country co ON c.CountryCode = co.Code
        JOIN countrylanguage cl ON co.Code = cl.CountryCode
        WHERE co.GovernmentForm NOT LIKE '%Republic%' AND 
        cl.IsOfficial = 'T' AND cl.Language != 'English' AND
        c.Population > 13000 AND c.Population < 500000
        ORDER BY RAND() 
        LIMIT 5
        """, cursor)

+-------------+---------------+--------------+-----------------------------------+--------------------+
| Name        | CountryCode   |   Population | GovernmentForm                    | OfficialLanguage   |
|-------------+---------------+--------------+-----------------------------------+--------------------|
| Qandahar    | AFG           |       237500 | Islamic Emirate                   | Dari               |
| al-Salimiya | KWT           |       130215 | Constitutional Monarchy (Emirate) | Arabic             |
| Kasukabe    | JPN           |       201838 | Constitutional Monarchy           | Japanese           |
| Kitami      | JPN           |       111295 | Constitutional Monarchy           | Japanese           |
| Gijón       | ESP           |       267980 | Constitutional Monarchy           | Spanish            |
+-------------+---------------+--------------+-----------------------------------+--------------------+


In [43]:
sql_exc("USE ro_employees", cursor)
sql_exc("SHOW TABLES", cursor)

+--------------------------+
| Tables_in_ro_employees   |
|--------------------------|
| departments              |
| dept_emp                 |
| dept_manager             |
| employees                |
| salaries                 |
| titles                   |
+--------------------------+


In [44]:
sql_exc("""
        SELECT COUNT(*) AS count_of_engineers_in_1985
        FROM salaries s
        JOIN employees e ON s.emp_no = e.emp_no
        JOIN titles t ON e.emp_no = t.emp_no
        WHERE t.title LIKE '%Engineer%' AND
        s.from_date BETWEEN '1985-01-01' AND '1986-01-01' AND
        s.to_date BETWEEN '1985-01-01' AND '1986-01-01'
        """, cursor)

+------------------------------+
|   count_of_engineers_in_1985 |
|------------------------------|
|                           39 |
+------------------------------+


In [45]:
sql_exc("""
        SELECT COUNT(1) AS count_of_production_employees
        FROM dept_emp de 
        JOIN departments d on de.dept_no = d.dept_no
        WHERE d.dept_name = 'Production' AND
        de.from_date BETWEEN '1985-01-01' AND '1992-01-01' AND
        de.to_date BETWEEN '1985-01-01' AND '1992-01-01'
        """, cursor)

+---------------------------------+
|   count_of_production_employees |
|---------------------------------|
|                            2517 |
+---------------------------------+


In [46]:
sql_exc("""
        SELECT e.emp_no, e.first_name, e.last_name, s.salary, t.title
        FROM employees e
        JOIN salaries s ON e.emp_no = s.emp_no
        JOIN titles t ON e.emp_no = t.emp_no
        WHERE s.to_date = '9999-01-01' AND
        t.to_date = '9999-01-01'
        ORDER BY RAND()
        LIMIT 20
        """, cursor)

+----------+--------------+----------------+----------+------------------+
|   emp_no | first_name   | last_name      |   salary | title            |
|----------+--------------+----------------+----------+------------------|
|   408799 | Vidya        | Marzano        |    46938 | Engineer         |
|    78180 | Jovan        | Stassinopoulos |    81589 | Senior Staff     |
|   213113 | Shigeichiro  | Hammerschmidt  |    59064 | Senior Engineer  |
|    22139 | Ayonca       | Goldhammer     |    53173 | Senior Engineer  |
|   473353 | Martins      | Vernadat       |    86658 | Senior Engineer  |
|    51281 | Mingsen      | Standera       |    86851 | Engineer         |
|   432960 | Youpyo       | Neiman         |    56340 | Senior Staff     |
|   432884 | Chaoyi       | Miculan        |   118735 | Senior Staff     |
|    54194 | Lijie        | Trystram       |    74909 | Senior Engineer  |
|   283494 | Gladys       | Percebois      |    79529 | Senior Engineer  |
|   434125 | Boriana     

In [328]:
sql_exc("SELECT * FROM employees LIMIT 20", cursor)

+----------+--------------+--------------+-------------+----------+-------------+
|   emp_no | birth_date   | first_name   | last_name   | gender   | hire_date   |
|----------+--------------+--------------+-------------+----------+-------------|
|    10001 | 1953-09-02   | Georgi       | Facello     | M        | 1986-06-26  |
|    10002 | 1964-06-02   | Bezalel      | Simmel      | F        | 1985-11-21  |
|    10003 | 1959-12-03   | Parto        | Bamford     | M        | 1986-08-28  |
|    10004 | 1954-05-01   | Chirstian    | Koblick     | M        | 1986-12-01  |
|    10005 | 1955-01-21   | Kyoichi      | Maliniak    | M        | 1989-09-12  |
|    10006 | 1953-04-20   | Anneke       | Preusig     | F        | 1989-06-02  |
|    10007 | 1957-05-23   | Tzvetan      | Zielinski   | F        | 1989-02-10  |
|    10008 | 1958-02-19   | Saniya       | Kalloufi    | M        | 1994-09-15  |
|    10009 | 1952-04-19   | Sumant       | Peac        | F        | 1985-02-18  |
|    10010 | 196

In [63]:
sql_exc("SELECT COUNT(*) FROM employees", cursor)

+------------+
|   COUNT(*) |
|------------|
|     300024 |
+------------+


In [71]:
conn, cursor = connect()

In [78]:
sql_exc("USE world", cursor)
sql_exc("SHOW TABLES", cursor)

+-------------------+
| Tables_in_world   |
|-------------------|
| city              |
| country           |
| countrylanguage   |
+-------------------+


In [81]:
sql_exc("DESC countrylanguage", cursor)

+-------------+---------------+--------+-------+-----------+---------+
| Field       | Type          | Null   | Key   | Default   | Extra   |
|-------------+---------------+--------+-------+-----------+---------|
| CountryCode | char(3)       | NO     | PRI   |           |         |
| Language    | char(30)      | NO     | PRI   |           |         |
| IsOfficial  | enum('T','F') | NO     |       | F         |         |
| Percentage  | decimal(4,1)  | NO     |       | 0.0       |         |
+-------------+---------------+--------+-------+-----------+---------+
